<a href="https://colab.research.google.com/github/aymanezzouhairi/dataminingProjet/blob/main/Projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import make_scorer, accuracy_score
from scipy.spatial.distance import cdist

# Charger les données
data = pd.read_csv('Test.csv')  # Remplacez par votre chemin
print("Aperçu des données :\n", data.head())

# Identifier les colonnes numériques et catégoriques
numeric_columns = data.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = data.select_dtypes(include=['object']).columns

# 1. Gestion des valeurs manquantes
# Remplacer les valeurs manquantes dans les colonnes numériques par la moyenne
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Remplacer les valeurs manquantes dans les colonnes catégoriques par le mode
for col in categorical_columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# 2. Encodage des variables catégoriques
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col].astype(str))

# Séparation des caractéristiques et de la cible
X = data.drop(columns=['ID', 'Segmentation'])
y = LabelEncoder().fit_transform(data['Segmentation'])

# 3. Normalisation des données avec Min-Max Scaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 4. Sélection des caractéristiques avec Chi-square test
chi2_selector = SelectKBest(chi2, k=8)  # Sélectionne les 8 meilleures caractéristiques
X_selected = chi2_selector.fit_transform(X_scaled, y)

# 5. Division des données pour la classification
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Implémentation de l'algorithme Firefly
class FireflyAlgorithm:
    def __init__(self, obj_function, n_dim, lb, ub, n_fireflies=20, max_iter=50, gamma=1.0, beta0=1.0, alpha=0.5):
        self.obj_function = obj_function
        self.n_dim = n_dim
        self.lb = np.array(lb)
        self.ub = np.array(ub)
        self.n_fireflies = n_fireflies
        self.max_iter = max_iter
        self.gamma = gamma
        self.beta0 = beta0
        self.alpha = alpha

    def optimize(self):
        fireflies = np.random.uniform(self.lb, self.ub, (self.n_fireflies, self.n_dim))
        fitness = np.apply_along_axis(self.obj_function, 1, fireflies)

        for t in range(self.max_iter):
            for i in range(self.n_fireflies):
                for j in range(self.n_fireflies):
                    if fitness[i] > fitness[j]:
                        distance = cdist([fireflies[i]], [fireflies[j]])[0][0]
                        beta = self.beta0 * np.exp(-self.gamma * distance ** 2)
                        fireflies[i] += beta * (fireflies[j] - fireflies[i]) + self.alpha * np.random.uniform(-1, 1, self.n_dim)
                        fireflies[i] = np.clip(fireflies[i], self.lb, self.ub)
                        fitness[i] = self.obj_function(fireflies[i])
        best_idx = np.argmin(fitness)
        return fireflies[best_idx], fitness[best_idx]

# Fonction objectif

def svm_objective(params):
    C, gamma = params
    model = SVC(C=C, kernel='rbf', gamma=gamma)
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring=make_scorer(accuracy_score))
    return -np.mean(scores)  # Négatif car on minimise

# Configuration et exécution de Firefly
lb = [0.1, 0.001]  # Limites inférieures pour C et gamma
ub = [10, 1]       # Limites supérieures pour C et gamma

firefly = FireflyAlgorithm(svm_objective, n_dim=2, lb=lb, ub=ub, max_iter=30)
best_params, best_score = firefly.optimize()

print(f"Meilleurs paramètres SVM trouvés : C={best_params[0]:.4f}, gamma={best_params[1]:.4f}")
print(f"Score de validation croisee correspondant : {best_score:.4f}")

# Modèle SVM optimisé
svm_model = SVC(C=best_params[0], kernel='rbf', gamma=best_params[1])
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Évaluation
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))

# Modèle Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
print("Rapport de classification pour Naive Bayes :\n", classification_report(y_test, y_pred_nb))

# Modèle Complement Naive Bayes
cnb_model = ComplementNB()
cnb_model.fit(X_train, y_train)
y_pred_cnb = cnb_model.predict(X_test)
print("Rapport de classification pour Complement Naive Bayes :\n", classification_report(y_test, y_pred_cnb))


Aperçu des données :
        ID  Gender Ever_Married  Age Graduated  Profession  Work_Experience  \
0  458989  Female          Yes   36       Yes    Engineer              0.0   
1  458994    Male          Yes   37       Yes  Healthcare              8.0   
2  458996  Female          Yes   69        No         NaN              0.0   
3  459000    Male          Yes   59        No   Executive             11.0   
4  459001  Female           No   19        No   Marketing              NaN   

  Spending_Score  Family_Size  Var_1 Segmentation  
0            Low          1.0  Cat_6            B  
1        Average          4.0  Cat_6            A  
2            Low          1.0  Cat_6            A  
3           High          2.0  Cat_6            B  
4            Low          4.0  Cat_6            A  
Meilleurs paramètres SVM trouvés : C=8.3788, gamma=0.1002
Score de validation croisee correspondant : -0.3579
SVM Classification Report:
               precision    recall  f1-score   support

    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m